# Predictor Class 

## Introduction 

So far in this notebook we are going to create a predictor class for 
detectin to what class a given tweet belongs to. 

the class 1 is Mexico and the class 2 is argentina. 






In [108]:
class robot_class_predictor: 
    def __init__(self):
        #here we have the global variables.so far we need 
        #1. the data frames 
        # the model architecture   
        #Noise removal, stop word removal, normalizing?
        self.vectorizer = None
        self.model = None
        self.df_class_1=None
        self.df_class_2=None
        self.concat_df=None
        self.tfidf=None
        
        
        
    def cleanString(self,s, special_chars = "\":,.@|ðÿœžðÿâœœïÿœžÿºÿÿœžÿ"):
        from nltk.tokenize import TweetTokenizer
        from nltk.corpus import stopwords



        for char in special_chars:
            s = s.replace(char, "")
        s = s.replace("\n", "")
        s = s.replace("https", "")
        s = self.scrub_words(s)
        tokenizer = TweetTokenizer()
        stop_words = set(stopwords.words('english'))
        cleaned_words = [w for w in tokenizer.tokenize(s) if w not in stop_words]
        return " ".join(cleaned_words)
    
    def cleanFrame(self,frame):
        frame['clean_tweet'] = frame.tweet.apply(self.cleanString)
        #methods for cleaning the tweets
        
        
    #preprocessing methods
    def scrub_words(self,text):
        # remove html markup
        import re
        text=re.sub("(<.*?>)","",text)
    
        #remove non-ascii and digits
        text=re.sub("(\\W|\\d)"," ",text)
  
        #remove whitespace
        text=text.strip()
        return text

        
        
    
        
    #The methods avaialabe to Load the information  
    def load_dataset_class_1(self):
        df_mexico = pd.read_csv('data/all_Tweets_2022-11-28(Mexico).csv')
        lnguage = 'en'
        df_mexico_1 = df_mexico.loc[df_mexico['lang']==lnguage]        
        t_1=pd.DataFrame({'tweet':list(df_mexico_1['text']),'class':0})
        #load the data to the class
        self.cleanFrame(t_1)
        self.df_class_1 = t_1
        t_1.to_csv('clean_data_c_1.csv')
        print('Data loaded successfully')

    def load_dataset_class_2(self):
        df_argentina = pd.read_csv('data/all_Tweets_2022-11-28(argentina).csv')
        lnguage = 'en'
        df_argentina_1 = df_argentina.loc[df_argentina['lang']==lnguage]        
        t_1=pd.DataFrame({'tweet':list(df_argentina_1['text']),'class':1})
        #load the data to the class
        self.cleanFrame(t_1)
        self.df_class_2 = t_1
        t_1.to_csv('clean_data_c_2.csv')

        print('Data loaded successfully')
        
        
    #=====================================================#
    def tweetToVec(self,tweet, row=0):
        words = tweet.split(" ")
        vec = np.zeros(self.tfidf.shape[1])
        for w in words:
            #print("including word " + w)
            if w in self.tfidf.columns:
                index = self.tfidf.columns.get_loc(w)
            #print(index, tfidf[w][row])
            vec[index] = self.tfidf[w][row]
        return vec
    
    
    def prepare_metadata(self):
        self.load_dataset_class_1()
        self.load_dataset_class_2()
        
        self.concat_df = pd.concat([self.df_class_1,self.df_class_2])
        
            #from sklearn.feature_extraction.text import TfidfVectorizer
        self.vectorizer = TfidfVectorizer()
        vectors = self.vectorizer.fit_transform(self.concat_df.tweet.tolist())
        feature_names = self.vectorizer.get_feature_names()
        dense = vectors.todense()
        denselist = dense.tolist()
        self.tfidf = pd.DataFrame(denselist, columns=feature_names)
        self.tfidf.head()
        
        
    
    
    
    # code for the training model 
    def prepare_data_for_tidf(self):
        
        self.load_dataset_class_1()
        self.load_dataset_class_2()
        
        self.concat_df = pd.concat([self.df_class_1,self.df_class_2])
        
            #from sklearn.feature_extraction.text import TfidfVectorizer
        self.vectorizer = TfidfVectorizer()
        vectors = self.vectorizer.fit_transform(self.concat_df.tweet.tolist())
        feature_names = self.vectorizer.get_feature_names()
        dense = vectors.todense()
        denselist = dense.tolist()
        self.tfidf = pd.DataFrame(denselist, columns=feature_names)
        self.tfidf.head()
        
        #print(self.tweetToVec(list(self.concat_df['tweet'])[0],self.tfidf))
        
        
        X = self.tfidf.values
        print(X.shape)
        print(np.ones(X.shape[0]).shape)
        #Adding bias term
        X = np.c_[np.ones(X.shape[0]), X]
        print(X.shape)
        
        Y = np.array(list(map(lambda x: min(x, 1),self.concat_df['class'])))
        X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.2, random_state=42)
        print(X.shape, X_train.shape, X_test.shape, len(y_train), len(y_test))
        
        
        training_data = X_train
        target_data = np.array(y_train)

        model = Sequential()
        model.add(Dense(X.shape[1], activation='relu'))
        model.add(Dropout(0.5))
        model.add(Dense(100, activation='relu'))
        model.add(Dropout(0.5))
        model.add(Dense(50, activation='relu'))
        model.add(Dense(1, activation='sigmoid'))
 
        model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
 
        model.fit(training_data, target_data, epochs=150)
 
        scores = model.evaluate(training_data, target_data)

        model.save('Training_clasification_v1.h5')

 
        print("\n%s: %.2f%%" % (model.metrics_names[1], scores[1]*100))
        print (model.predict(training_data).round())
        
        
        
        
        ############################3
    def predict_new(self,tweet):
        model = tf.keras.models.load_model('Training_clasification_v1.h5')
        #vectorize = self.tweetToVec(tweet,row=0)
        tweet2 = [tweet,'a']
        vectors = self.vectorizer.transform(tweet2)
        feature_names = self.vectorizer.get_feature_names()
        dense = vectors.todense()
        denselist = dense.tolist()
        tfidf = pd.DataFrame(denselist, columns=feature_names)
        X=tfidf.values
        
        #X = vectorize
        print(X.shape)
        print(np.ones(X.shape[0]).shape)
        #Adding bias term
        X = np.c_[np.ones(X.shape[0]), X]
        print(X.shape)

        val = model.predict(X).round()
        print(model.predict(X).round())

        return val

            
                                                     
        
        
        

In [109]:
import pandas as pd
import seaborn as sns
import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords
from nltk.tokenize import TweetTokenizer
from nltk.stem import PorterStemmer
import re
from sklearn.model_selection import train_test_split
#from sklearn.preprocessing.label import MultiLabelBinarizer
from sklearn.preprocessing._label import MultiLabelBinarizer
from sklearn.feature_extraction.text import TfidfVectorizer

import tensorflow as tf
print(tf.__version__)
import numpy as np 
import numpy as np
from keras.models import Sequential
from keras.layers.core import Dense
from keras.layers import Dropout


2.10.0


[nltk_data] Downloading package stopwords to
[nltk_data]     /home/augusto/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [110]:
wall_e = robot_class_predictor()

In [111]:
wall_e.prepare_metadata()

Data loaded successfully
Data loaded successfully


/home/augusto/anaconda3/lib/python3.9/site-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)


In [112]:
df_mexico = pd.read_csv('data/all_Tweets_2022-11-28(Mexico).csv')
df_mexico.head()
#df_mexico.loc[df_mexico['lang']=='en']
lnguage = 'en'
# saving the data for mexico 
t_1=pd.DataFrame({'tweet':list(df_mexico.loc[df_mexico['lang']==lnguage]['text']),'senti':0})
#t_1 = df_mexico['text']
#t_1['class'] = 0
t_1




,tweet,senti
0,📊Números de la #SeleccionMexicana 🇲🇽 en Mundia...,0
1,The joy of seeing your hero perform at the #FI...,0
2,The joy of seeing your hero perform at the #FI...,0
3,PAY US TO HELP IN YOUR DUE\n\nHomework\nAssign...,0
4,PAY US TO HELP IN YOUR DUE\n\nHomework\nAssign...,0
5,PAY US TO HELP IN YOUR DUE\n\nHomework\nAssign...,0
6,PAY US TO HELP IN YOUR DUE\n\nHomework\nAssign...,0
7,@miseleccionmxEN @miseleccionmx \n$8.8 Million...,0
8,Playing cricket in parkikg lot. LilLoquito fou...,0
9,Okay last football tweet: Mexico’s football fe...,0


In [117]:
ect = wall_e.predict_new(list(t_1['tweet'])[8])
ect[0]


(2, 1259)
(2,)
(2, 1260)
1/1 [==============================] - 0s 26ms/step


/home/augusto/anaconda3/lib/python3.9/site-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)


[[0.]
 [1.]]


array([0.], dtype=float32)

In [116]:
ect[0]

array([0.], dtype=float32)